In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# book_test 데이터프레임 정의 (단일 URL)
book_test = pd.DataFrame({
    'd_url': ['https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=333420348']
})

# 함수화. 반환값은 되도록 pandas 의 Series 로
def get_aladin_book_details(detail_url: str) -> pd.Series:
    try:
        url = detail_url
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')

        # Sales Point
        sp = soup.find('div', 'Ere_fs15 Ere_ht18')
        sp = sp.find('div', style='display:inline-block;')
        sales_point = sp.find('strong').text if sp else None

        # Price
        price_tag = soup.find('em', 'Ere_fs24 samnung_pink')
        price = price_tag.text if price_tag else None

        # Page Count
        page_count = None
        info_1 = soup.find('div', 'conts_info_list1')
        if info_1:
            for li in info_1.find_all('li'):
                if li.text.endswith('쪽'):
                    page_count = li.text[:-1]
                    break

        # Category
        category = None
        info_2 = soup.find('div', 'conts_info_list2')
        if info_2:
            category_line = info_2.find_all('li')[0]
            category_tag = category_line.find_all('a')[2] if category_line else None
            category = category_tag.text if category_tag else None

        # Review Count
        review_count = None
        review_div = soup.find('div','info_list Ere_fs15 Ere_ht18')
        if review_div:
            review_a_tags = review_div.find_all('a')
            if len(review_a_tags) > 3:
                review_count = review_a_tags[3].text.replace('리뷰(', '').replace(')', '')
                
        # Average Rating
        rating_tag = soup.find('a', 'Ere_sub_pink Ere_fs16 Ere_str')
        average_rating = rating_tag.text.strip() if rating_tag else None

        return pd.Series({
            "sales_point": sales_point,
            "price": price,
            "page_count": page_count,
            "category": category,
            "review_count": review_count,
            "average_rating": average_rating
        })
    
    except :
        return pd.Series({
            "sales_point": None,
            "price": None,
            "page_count": None,
            "category": None,
            "review_count": None,
            "average_rating": None
        })
book_test[['sales_point', 'price', 'page_count', 'category', 'review_count', 'average_rating']] = book_test['d_url'].apply(get_aladin_book_details)
book_test

,d_url,sales_point,price,page_count,category,review_count,average_rating
0,https://www.aladin.co.kr/shop/wproduct.aspx?It...,61,"11,700",92,동화/명작/고전,0,0.0
